# Required modules

In [2]:
import attr
import random
import typing

# Basics

In its most basic form, `attr` resembles the standard Python `dataclass`.

In [11]:
@attr.s
class Person():
    id = attr.ib()
    first_name = attr.ib()
    last_name = attr.ib()
    age = attr.ib()

Objects of this class have four attributes.  They can be initialized by using keyword arguments in the constructor.

In [12]:
p1 = Person(id=123, first_name='Albert', last_name='Einstein', age=67)

`attr` automatically generates a nice `__repr__` for you.

In [13]:
p1

Person(id=123, first_name='Albert', last_name='Einstein', age=67)

Attributes behave as if they were defined through Python's standard decorators.

In [14]:
p1.age

67

In [15]:
p1.age = 48

In [16]:
p1.age

48

## Private attributes

In [27]:
@attr.s
class PublicPrivate():
    value = attr.ib()
    _mine = attr.ib()

Note that to initialize the private attribute `_mine`, you can leave off the leading underscore.

In [28]:
obj = PublicPrivate(value=17, mine=12)

In [29]:
obj

PublicPrivate(value=17, _mine=12)

Note that the private attribute can not be accessed without the undersocre.

In [30]:
try:
    print(obj.mine)
except:
    print('No way')

No way


You can initialize private attributes yourself by specifying the default value and setting `init` to `False`.

In [29]:
@attr.s
class PublicPrivate():
    value = attr.ib()
    _mine = attr.ib(init=False, default=42)

In [30]:
PublicPrivate(3)

PublicPrivate(value=3, _mine=42)

## Mutable attribute initialization

You can initialize attributes yourself in non-trivial ways.  The `_values` attribute will be initialized by the return value of the method `init_values`.

In [9]:
@attr.s
class Guessing:
    nr_values = attr.ib()
    _values = attr.ib(init=False)
    @_values.default
    def init_values(self):
        values = set()
        while len(values) < self.nr_values:
            values.add(random.choice(list(range(10))))
        return values
    
    def has_value(self, value):
        return value in self._values

In [10]:
guess1 = Guessing(3)

In [11]:
guess1

Guessing(nr_values=3, _values={8, 4, 7})

In [12]:
guess2 = Guessing(3)

In [13]:
guess2

Guessing(nr_values=3, _values={8, 9, 3})

In [14]:
guess1.has_value(4)

True

In [15]:
guess2.has_value(4)

False

Note that mutable attributes have to be defined either using the method above, or using `attr.Factory`.  The class below illustrates what happens if you are naive about this.

In [16]:
@attr.s
class Naive:
    values = attr.ib(init=False, default=list())

In [17]:
obj1 = Naive()

In [18]:
obj2 = Naive()

In [19]:
obj1.values.append(5)

In [20]:
obj2

Naive(values=[5])

This may be surprising until you realize that the list is instantiated when the class definition is executed, and hence is the same for all objects instantiated for that class.

In [31]:
@attr.s
class NotSoNaive:
    values = attr.ib(init=False, factory=list)

In [32]:
obj1 = NotSoNaive()

In [33]:
obj2 = NotSoNaive()

In [34]:
obj1.values.append(5)

In [35]:
obj1

NotSoNaive(values=[5])

In [36]:
obj2

NotSoNaive(values=[])

## Validators

You can add validators to attributes, e.g., to check whether a value is in a certain range.

In [82]:
@attr.s
class Guessing:
    nr_values = attr.ib(type=int)
    @nr_values.validator
    def nr_values_validate(self, attribute, value):
        if value < 1 or value > 10:
            raise ValueError('number of values should be between 1 and 10')
    _values = attr.ib(init=False)
    @_values.default
    def init_values(self):
        values = set()
        while len(values) < self.nr_values:
            values.add(random.choice(list(range(10))))
        return values
    
    def has_value(self, value):
        return value in self._values

In [66]:
try:
    Guessing(-1)
except ValueError as error:
    print(error)

number of values should be between 1 and 10


Note that default factories are executed before any validators, which explains why the following doesn't result in a validation error.

In [76]:
try:
    Guessing('abc')
except TypeError as error:
    print(error)

'<' not supported between instances of 'int' and 'str'


You can add type information to attributes, either using Python's type hints, or `attr.ib`'s keyword argument.

In [77]:
@attr.s
class Numbers:
    x = attr.ib(type=float)
    y : str = attr.ib()

However, don't get excited, this will only help static type checkers such as mypy, no runtime checks are added.

In [78]:
Numbers(x='abc', y=3)

Numbers(x='abc', y=3)

## Converters

You can specify a converter for an attribute value.  This will be invoked after the default factory, but before the validator.

In [83]:
@attr.s
class Guessing:
    nr_values : int = attr.ib(converter=int)
    @nr_values.validator
    def nr_values_validate(self, attribute, value):
        if value < 1 or value > 10:
            raise ValueError('number of values should be between 1 and 10')
    _values : typing.List[int] = attr.ib(init=False)
    @_values.default
    def init_values(self):
        values = set()
        while len(values) < self.nr_values:
            values.add(random.choice(list(range(10))))
        return values
    
    def has_value(self, value):
        return value in self._values

In [84]:
try:
    Guessing('abc')
except ValueError as error:
    print(error)

invalid literal for int() with base 10: 'abc'


In [85]:
Guessing('4')

Guessing(nr_values=4, _values={0, 9, 2, 5})

In [88]:
try:
    Guessing('-9')
except ValueError as error:
    print(error)

number of values should be between 1 and 10


## Post-init hook

Sometimes you want to do initialization based on the values of the attributes passed to the constructor.  You would typically do that in the `__init__` method.  To do similar things for `attrs` classes, you can use the `__attrs_post_init__` method.

In [7]:
@attr.s
class Guessing:
    nr_values : int = attr.ib(converter=int)
    _values : typing.List[int] = attr.ib(init=False)

    @nr_values.validator
    def nr_values_validate(self, attribute, value):
        if value < 1 or value > 10:
            raise ValueError('number of values should be between 1 and 10')

    def __attrs_post_init__(self):
        self._values = set()
        while len(self._values) < self.nr_values:
            self._values.add(random.choice(list(range(10))))
    
    def has_value(self, value):
        return value in self._values

In [10]:
guessing = Guessing(4)
print(guessing)
for guess in random.choices(range(10), k=3):
    print(f'trying {guess}')
    if guessing.has_value(guess):
        print(f'hurray for {guess}!')
    else:
        print(f'no luck with {guess}')

Guessing(nr_values=4, _values={0, 2, 5, 6})
trying 7
no luck with 7
trying 4
no luck with 4
trying 8
no luck with 8


## Comparisons

By default, `attrs` will construct comparison methods for you, i.e., `__eq__`, `__neq__`, but also `__lt__` and so on.

In [18]:
@attr.s
class Person:
    lastname: str = attr.ib()
    firstname: str = attr.ib()
    age: int = attr.ib()
        
    @age.validator
    def age_validator(self, attribute, value):
        if value < 0 or value >= 130:
            raise ValueError(f'age should be between 0 and 130')

The natural order for objects of this class is lexicographic on lastname, firstname, and the numeric on age.  Note that the declaration order of the attributes matters and determines the final sort order.

In [24]:
people = [
    Person(firstname='Alice', lastname='Zosimo', age=43),
    Person(firstname='Aaron', lastname='Leibovitch', age=31),
    Person(firstname='Robert', lastname='Leibovitch', age=49),
    Person(firstname='Alice', lastname='Zosimo', age=25),
]
people

[Person(lastname='Zosimo', firstname='Alice', age=43),
 Person(lastname='Leibovitch', firstname='Aaron', age=31),
 Person(lastname='Leibovitch', firstname='Robert', age=49),
 Person(lastname='Zosimo', firstname='Alice', age=25)]

We can no use Python's `sorted` function to sort the people in this list according to lastname, firstname and age.

In [23]:
sorted(people)

[Person(lastname='Leibovitch', firstname='Aaron', age=31),
 Person(lastname='Leibovitch', firstname='Robert', age=49),
 Person(lastname='Zosimo', firstname='Alice', age=25),
 Person(lastname='Zosimo', firstname='Alice', age=43)]

Equality is also based on the object's attributes.

In [25]:
alice1 = Person(firstname='Alice', lastname='Zosimo', age=43)
alice2 = Person(firstname='Alice', lastname='Zosimo', age=43)
alice3 = Person(firstname='alice', lastname='zosimo', age=43)

`alice1` has the same attributes as `alice2`, but they are distinct objects nevertheless.

In [26]:
alice1 == alice2

True

In [27]:
alice1 is alice2

False

`alice3` is different from `alice1`

In [28]:
alice1 == alice3

False

You can exclude attrubutes from being used to compare objects for equality by adding the argument `eq=False` to the `attr.ib` function call.  Similarly, you can exclude an attribute from order comparisons by setting `order` to `False`.

## Hashing

Mutable objects are not hashable, which means they can not be stored in sets, or used as keys in dictionaries.

In [32]:
try:
    people = {
        Person(firstname='Alice', lastname='Zosimo', age=43),
        Person(firstname='Aaron', lastname='Leibovitch', age=31),
    }
except Exception as error:
    print(error)

unhashable type: 'Person'


One options is to make the objects unmutable.

In [33]:
@attr.s(frozen=True)
class Person:
    lastname: str = attr.ib()
    firstname: str = attr.ib()
    age: int = attr.ib()
        
    @age.validator
    def age_validator(self, attribute, value):
        if value < 0 or value >= 130:
            raise ValueError(f'age should be between 0 and 130')

In [53]:
people = {
    Person(firstname='Alice', lastname='Zosimo', age=43),
    Person(firstname='Aaron', lastname='Leibovitch', age=31),
}
people

{Person(lastname='Leibovitch', firstname='Aaron', age=31),
 Person(lastname='Zosimo', firstname='Alice', age=43)}

However, this implies you can not modify the objects.

In [55]:
person = people.pop()
print(person)
try:
    person.age = 12
except AttributeError:
    print('the objects is frozen')
print(person)

Person(lastname='Leibovitch', firstname='Aaron', age=31)
the objects is frozen
Person(lastname='Leibovitch', firstname='Aaron', age=31)


This is of course fine if you do not need mutable ojbects.  The alternative is to hash based on object identity.

In [56]:
@attr.s(eq=False)
class Person:
    lastname: str = attr.ib()
    firstname: str = attr.ib()
    age: int = attr.ib()
        
    @age.validator
    def age_validator(self, attribute, value):
        if value < 0 or value >= 130:
            raise ValueError(f'age should be between 0 and 130')

However, this implies that objects will be distinct, regardless of the values of their attributes.

In [58]:
people = {
    Person(firstname='Alice', lastname='Zosimo', age=43),
    Person(firstname='Aaron', lastname='Leibovitch', age=31),
    Person(firstname='Robert', lastname='Leibovitch', age=49),
    Person(firstname='Alice', lastname='Zosimo', age=43),
}
people

{Person(lastname='Leibovitch', firstname='Aaron', age=31),
 Person(lastname='Leibovitch', firstname='Robert', age=49),
 Person(lastname='Zosimo', firstname='Alice', age=43),
 Person(lastname='Zosimo', firstname='Alice', age=43)}

In [59]:
alice1 = Person(firstname='Alice', lastname='Zosimo', age=43)
alice2 = Person(firstname='Alice', lastname='Zosimo', age=43)
alice3 = Person(firstname='alice', lastname='zosimo', age=43)

`alice1` has the same attributes as `alice2`, but they are distinct objects nevertheless.

In [60]:
alice1 == alice2

False

In [61]:
alice1 is alice2

False

Of course, you can still impliment your own hash method and comparison methods if required.

# Example

We will implement the `Point` class that was sued as a running example.

In [62]:
from math import sqrt

In [80]:
@attr.s
class Point:
    x: float = attr.ib(converter=float)
    y: float = attr.ib(converter=float)
        
    def distance(self, other):
        return sqrt((self.x - other.x)**2 + (self.y - other.y)**2)
    
    @property
    def coords(self):
        return (self.x, self.y)
    
    @coords.setter
    def coords(self, value):
        self.x, self.y = value

In [68]:
p1, p2 = Point(3.1, 5), Point('3.2', 1.9)
p1, p2

(Point(x=3.1, y=5.0), Point(x=3.2, y=1.9))

Note the differences with the original "pure Python" implementation:
* `x` and `y` are public attributes, and
* values assigned to the attributes are not validated.

In [71]:
p1.x = 'abc' 
p1

Point(x='abc', y=5.0)

If you want that level of control, the advantages of using `attrs` start to decrease considerably.

In [74]:
p1.coords = 17.1, 12.5
p1

Point(x=17.1, y=12.5)

## Inheritance

Inheritance simply works as expected.  Methods and attributes can be added to subclasses.

In [75]:
@attr.s
class PointMass(Point):
    mass: float = attr.ib(converter=float)
    
    def __attr_pre_init__(self)

In [78]:
p1 = PointMass(1.2, 2.3, 5.0)
p1

PointMass(x=1.2, y=2.3, mass=5.0)

In [82]:
p2 = Point(3.2, 4.1)

In [83]:
p1.distance(p2)

2.690724809414742